In [ ]:
import dask.dataframe as dd
from dask.distributed import Client
import json
import dask
import numpy as np
import re
# import graphviz
import pandas as pd
import random

In [ ]:
client=Client('172.31.42.114:8786')
client
# client.scheduler_info()
# #dask.config.config

In [ ]:
products = dd.read_csv("small_products_Release.csv")
reviews = dd.read_csv("small_user_reviews_Release.csv")

In [ ]:
products.head(4)

In [ ]:
reviews.head(1)

In [ ]:
products.shape[0].compute()

In [ ]:
products.head()


In [ ]:
reviews.isna().mean().compute()

In [ ]:
products.isna().mean().compute()

In [ ]:
subproducts = products[['asin', 'price']]
subreviews = reviews[['asin', 'overall']]

#optimise with left and not index on products
joined = subproducts.join(subreviews.set_index("asin"), on="asin", how='inner')
#joined.head()
pcor = joined.corr()['overall']
val = pcor.compute()['price']
val

In [ ]:
price_prod = products['price'].describe().compute()
#check if min is actuatly returning empty
# [price_prod['mean'], price_prod['std'], price_prod['50%'], price_prod['min'], price_prod['max']]
price_prod[['mean', 'std', '50%', 'min', 'max']]

In [ ]:
def f(x):
    if isinstance(x, float):
        return x
    else:
        return re.search("\'.*?\'",x).group().strip("'")

numCat = products.assign(categories = lambda x: x.categories.apply(
    f,
    meta=('categories', 'str'))
                        )

numCat = numCat.groupby("categories").count()['asin']
q4 = numCat.compute()[1:]

#[1:]this removes the nan catagory

In [ ]:
# #Q5

# subR = reviews[['asin']].drop_duplicates().set_index('asin')

# subP = products.set_index('asin')

# iner = subR.join(subP, on="asin", how='inner').shape[0]

# left = subR.join(subP, on="asin", how='left').shape[0]

# # print(iner)
# # print(left)

# # out = (iner == left).compute()

# # out
# iner.head()
# #products.set_index('asin').index
# #index.difference(products.set_index('asin').index)

In [ ]:
rev_data = ['A' + str(i) for i in range(170)]
prod_data = ['A' + str(i) for i in range(160)]
rev_data.extend(['A' + str(i) for i in range(20)])


test_reviews = dd.from_pandas(
    pd.DataFrame(
        {'asin':rev_data}
    ),
    npartitions = 16
)
test_products = dd.from_pandas(
    pd.DataFrame(
        {'asin':prod_data}
    ),
    npartitions = 16
)
test_reviews.tail(10)
# test_products.head(10)


In [ ]:
# # Q5 (without index)
subR = reviews[['asin']]

subP = products[["asin"]]
subP["temp"] = 1

left = dd.merge(subR, subP, how = "left")

left.temp.isna().any().compute() + 0

In [38]:
from string import ascii_uppercase
from string import digits
import random
import numpy as np
random.seed(100)

In [41]:
def rand_asin():
    return ''.join(random.choice(ascii_uppercase + digits) for i in range(10))
def rand_related(X, n):
    return str(random.sample(X, k = random.randint(1,n)))

prod_data = [rand_asin() for i in range(161)]
rev_data = [rand_related(prod_data, 10) for i in range(160)]

rev_data.append(np.nan)

test_products = dd.from_pandas(
    pd.DataFrame(
        {'asin':prod_data,
         'related':rev_data}
        
    ),
    npartitions = 16
)

test_products.tail(10)

,asin,related
154,E3JW4QT9QP,"['LRZZ37INOC', 'FQQ1RU3CVV', 'RQHW53ZTR1', 'JX..."
155,F4W9M858ZI,"['NNP5GMYQBD', 'TNW8RTOUO1', 'RBZ4MVE2EJ']"
156,3IZ5FZFV1C,"['VJW8SHCUSD', 'PJHL93WH3D', '0MG99JIK00', 'WI..."
157,AMOJ6FZWMK,"['JX3ONFY69F', '7GGPAXKASG', 'GNQ9ED1I18', 'Y8..."
158,KC5XTFRUFU,"['NJ9FDVWT8R', '289ONDKSB6', '9K91DMMLMU', 'UI..."
159,IPRAF3Y8M0,"['V1174LXD13', 'XKH18OKYVC']"
160,IKGFOK205T,NaN


In [42]:
def g(ser):
    lst = []
    def parse(related):
        if (not pd.isnull(related)):
            lst.extend(re.findall("[A-Z0-9]{10}", related))

    ser.apply(parse)
    return pd.Series(lst)

def f(df):
    return df.apply(g)

subR = test_products[["related"]]
metadata = {"related":"str"}
res = subR.map_partitions(f, meta = metadata)
res = res.rename(columns={"related": "asin"})
res = res.drop_duplicates(split_out=16)

subA = test_products[["asin"]]
subA["tempA"] = 1
res2 = res
res2["tempR"] = 1
jn = subA.merge(res2, how='outer', on='asin')

((jn.tempA.isna().any() + jn.tempR.isna().any()).compute() > 0) + 0


0

In [22]:
jn.compute()

,asin,tempA,tempR
0,IMAPVJW8SH,1,1
1,30ITT11S32,1,1
2,ZTR14E94UZ,1,1
3,K2DROHEOTX,1,1
4,ESYMM9892N,1,1
...,...,...,...
5,PJN0LVAZVJ,1,1
6,3ED0P81JWL,1,1
7,I6T74PBLJP,1,1
8,58ZI3IZ5FZ,1,1


In [19]:
jn.tempA.isna().any().compute()

False

In [18]:
jn.tempR.isna().any().compute()

True

In [ ]:
#Q6

subP = products

ser = subP["asin"]

def parse(related):
    return "|".join(re.findall("[A-Z0-9]{10}", related))

def f(val):
    asins = parse(val)
    return ser.str.contains("|".join(asins)).any()
    
def g(df):
    df['related'].apply(f,meta="str")

#f({"related":"0000143561"}).compute()
#f({"related":"B0036FO6SI"}).compute()

subP.assign(related = g,meta="str").compute()



In [ ]:
subP = products.set_index("asin")


def parse(related):
    return re.findall("[A-Z0-9]{10}", related)

def f(df):
    asins = []
    for row in df.iterrows():
        
        asins.append([1])
    
    ret = pd.DataFrame({"asins" : asins})
    return ret
    

subP.map_partitions(f,  meta="str").compute()


In [ ]:
# Q6 new



def g(ser):
    lst = []
    def parse(related):
        if (not pd.isnull(related)):
            lst.extend(re.findall("[A-Z0-9]{10}", related))
    
    ser.apply(parse)
    
    return pd.Series(lst)

def f(df):
    return df.apply(g)

In [ ]:
subR = products[["related"]]

metadata = {"related":"str"}

res = subR.map_partitions(f, meta = metadata)

res = res.rename(columns={"related": "asin"})

res = res.drop_duplicates(split_out=4)#10461397

#res.npartitions

#res.shape[0].compute()

In [ ]:
subA = products[["asin"]]

#subA = subA.set_index("asin")

subA["tempA"] = 1

res2 = res

#res2 = res2.set_index("asin")

res2["tempR"] = 1

jn = subA.merge(res2, how='outer', 
                left_index = True, 
                right_index = True)#on="asin")

In [ ]:
# jn.dask

In [ ]:
(jn.tempA.isna().any() + jn.tempR.isna().any()).compute() >0

In [ ]:
jn.tempA.isna().any().compute()

## FINAL CODE

In [1]:
import dask.dataframe as dd
from dask.distributed import Client
import json
import dask
import re
import pandas as pd

products = dd.read_csv("products_Release.csv")
reviews = dd.read_csv("user_reviews_Release.csv")

client=Client('172.31.32.105:8786')
client.restart()

# Q1
q1_reviews = reviews.isna().mean().compute() * 100
q1_products = products.isna().mean().compute() * 100


# Set up for Q1-Q6
# product_metadata = {'price':float,  
#                     'categories':str, 
#                     'asin':str,
#                     'related':str}
products = dd.read_csv("products_Release.csv", 
                        usecols=['price',  
                                'categories', 
                                'asin',
                                'related'
                                ],
                      )
# reviews_metadata = {'asin':str, 
#                     'overall':float}
reviews = dd.read_csv("user_reviews_Release.csv", 
                      usecols=['asin', 'overall'],
                      )
# products.drop(
#     [
#         "salesRank",
#         "imUrl",
#         "title",
#         "description",
#         "brand"
#     ], axis=1)

# reviews.drop(
#     [
#         "reviewerID",
#         "reviewerName",
#         "helpful",
#         "reviewText",
#         "summary",
#         "unixReviewTime",
#         "reviewTime"
#     ], axis=1)

bag1 = {}

# Q2
subproducts = products[['asin', 'price']]
subreviews = reviews[['asin', 'overall']]

#optimise with left and not index on products
# joined = subproducts.join(subreviews, 
#                           on="asin", 
#                           how='inner')
joined = dd.merge(subproducts, subreviews, on = 'asin', how = 'inner')

#joined.head()
bag1['q2'] = joined.corr()['overall']#.compute()['price']

# Q3
price_prod = products['price'].describe()
bag1['q3'] = price_prod#.compute()[['mean', 'std', '50%', 'min', 'max']]

# Q4
def get_supercat(x):
    if isinstance(x, float):
        return x
    else:
        return re.search("\'.*?\'",x).group().strip("'")

numCat = products.assign(categories = lambda x: x.categories.apply(
    get_supercat,
    meta=('categories', 'str'))
                        )

numCat = numCat.groupby("categories").count()['asin']
bag1['q4'] = numCat#.compute()[1:]


# Q5
subR = reviews[['asin']]
subP = products[["asin"]]
subP["temp"] = 1

left = dd.merge(subR, subP, how = "left")
bag1['q5'] = left.temp.isna().any()#.compute() + 0


# Q6
def g(ser):
    lst = []
    def parse(related):
        if (not pd.isnull(related)):
            lst.extend(re.findall("[A-Z0-9]{10}", related))

    ser.apply(parse)
    return pd.Series(lst)

def f(df):
    return df.apply(g)

subR = products[["related"]]
metadata = {"related":"str"}
res = subR.map_partitions(f, meta = metadata)
res = res.rename(columns={"related": "asin"})
res = res.drop_duplicates(split_out=16)

subA = products[["asin"]]
subA["tempA"] = 1
res2 = res
res2["tempR"] = 1
jn = subA.merge(res2, how='outer', on='asin')

q6 = (jn.tempA.isna().any() + jn.tempR.isna().any()).compute() > 0

temp1 = dask.compute(bag1)


In [2]:
temp1 = temp1[0]  
q2 = temp1['q2']['price']
q3 = temp1['q3'][['mean', 'std', '50%', 'min', 'max']]
q4 = temp1['q4'][1:].sort_index(ascending=True, kind="mergesort").sort_values(ascending=False, kind="mergesort")
q5 = temp1['q5'] + 0
q6 = q6 + 0

In [3]:
q1_reviews

reviewerID        0.000000
asin              0.000000
reviewerName      3.292575
helpful           0.000000
reviewText        0.021570
overall           0.000000
summary           0.001085
unixReviewTime    0.000010
reviewTime        0.000000
dtype: float64

In [4]:
q1_products

asin            0.000000
salesRank      26.015663
imUrl           1.795239
categories      0.798038
title          15.192672
description    41.717847
price          35.703404
related        29.039236
brand          81.684125
dtype: float64

In [5]:
q2

-0.012692858217330167

In [6]:
q3

mean     34.937356
std      71.263692
50%      19.600000
min       0.000000
max     999.990000
Name: price, dtype: float64

In [7]:
q4

categories
Books                                       2369910
Clothing, Shoes & Jewelry                   1435868
Sports & Outdoors                            529989
Electronics                                  495476
CDs & Vinyl                                  491713
                                             ...   
Publishers                                        2
Gospel                                            2
Celebrate your Birthday with Nickelodeon          2
Collectible Coins                                 1
#508510                                           1
Name: asin, Length: 82, dtype: int64

In [8]:
q5

1

In [9]:
q6

1